In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
cd "/content/gdrive/MyDrive/Colab Notebooks/myhuggingface/sequenceclassification"

/content/gdrive/MyDrive/Colab Notebooks/myhuggingface/sequenceclassification


In [3]:
pip install transformers kedro datasets python-box h11

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.6 MB/s 
     |████████████████████████████████| 249 kB 69.8 MB/s 
     |████████████████████████████████| 362 kB 73.2 MB/s 
     |████████████████████████████████| 3.0 MB 57.0 MB/s 
     |████████████████████████████████| 58 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 65.3 MB/s 
     |████████████████████████████████| 6.6 MB 49.8 MB/s 
     |████████████████████████████████| 101 kB 10.7 MB/s 
     |████████████████████████████████| 636 kB 66.2 MB/s 
     |████████████████████████████████| 51 kB 417 kB/s 
     |████████████████████████████████| 64 kB 3.9 MB/s 
     |████████████████████████████████| 202 kB 75.6 MB/s 
     |████████████████████████████████| 133 kB 75.4 MB/s 
     |████████████████████████████████| 181 kB 79.3 MB/s 
     |████████████████████████████████| 185 kB 70.8 MB/s 
     |████████████████████████████████

In [4]:
!python --version

Python 3.7.13


In [8]:
# !kedro run
!kedro run

2022-07-06 16:14:05,144 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-07-06 16:14:05,165 - kedro.framework.session.session - WARNING - Unable to git describe /content/gdrive/MyDrive/Colab Notebooks/myhuggingface/sequenceclassification
2022-07-06 16:14:05,181 - kedro.framework.session.session - INFO - ** Kedro project sequenceclassification
2022-07-06 16:14:05,370 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.
/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packages/flatbuffers/compat.py:19:

## Make catalog available

In [16]:
%load_ext kedro.extras.extensions.ipython
%reload_kedro /content/gdrive/MyDrive/Colab Notebooks/myhuggingface/sequenceclassification

The kedro.extras.extensions.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.extras.extensions.ipython
2022-07-06 16:38:26,142 - kedro.extras.extensions.ipython - INFO - Updated path to Kedro project: /content/gdrive/MyDrive/Colab Notebooks/myhuggingface/sequenceclassification
2022-07-06 16:38:26,303 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-07-06 16:38:26,333 - kedro.framework.session.session - WARNING - Unable to git describe /content/gdrive/MyDrive/Colab Notebooks/myhuggingface/sequenceclassification
2022-07-06 16:38:26,398 - kedro.extras.extensions.ipython - INFO - ** Kedro project sequenceClassification
2022-07-06 16:38:26,403 - kedro.extras.extensions.ipython - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


In [21]:
model = catalog.load('model')
small_eval_dataset = catalog.load('small_eval_dataset')

2022-07-06 16:41:22,149 - kedro.io.data_catalog - INFO - Loading data from `model` (PickleDataSet)...
2022-07-06 16:41:22,988 - kedro.io.data_catalog - INFO - Loading data from `small_eval_dataset` (PickleDataSet)...


# Put data into dataloader

In [ ]:
from torch.utils.data import DataLoader

eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

list(eval_dataloader)[0]

# Calc metrics by hand

In [27]:
!nvidia-smi

Wed Jul  6 16:42:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    27W /  70W |   1462MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [58]:
from datasets import load_metric
metric = load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    print(batch)

    batch = {k: v for k, v in batch.items() if k in [ 'input_ids']}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'label': tensor([2, 4, 1, 4, 3, 4, 2, 3]), 'text': ['Kabuto is your run-of-the-mill Japanese Steakhouse. Different stations with chefs slinging shrimp tails around the communal dining areas like it\'s a lunchtime magic show. Always a plethora of laughs and gags going around the group. \\n\\nThis place is great for lunch. $9 and 30 minutes and you\'re out the door. Uhhh...If I\'m craving a salad with ginger dressing, which I always am, (you do too. admit it) fried rice, steak, shrimp and white sauce (DUDE) then Kabuto is king of lunch options in my book. Always super clean and full of kindhearted staff. The parking lot is super difficult to get in and out of though. 51 traffic at lunch is a beast. Good luck getting stuck behind someone trying to cut across traffic at 12pm on a weekday. It\'s murder. This place would greatly benefit from another exit/entrance or a stoplight. Here\'s hoping....\\n\\nYou can\'t really shake a stick at balanced lunch when you can have soup or a salad, vegg

AttributeError: ignored

In [61]:
blub = {k: v for k, v in batch.items()}

In [62]:
blub

{'input_ids': [tensor([101, 101, 101, 101, 101, 101, 101, 101]),
  tensor([14812, 19383, 12008,  1188,   146,  3930,  1284,  6682]),
  tensor([16442,  1303, 27788,  1110,  1274, 13991,  3523,  3537]),
  tensor([1186, 1111, 6434,  170,  112,  117, 1303, 1171]),
  tensor([1110, 1275, 2094, 2712,  189, 1147, 1103, 1118]),
  tensor([ 1240,  1552,   102,  4041,  2385,  2094,  1480, 11637]),
  tensor([1576, 1105,    0, 1114, 1243, 1110, 1196, 6737]),
  tensor([  118,  6218,     0,   170,  1142,  6976,  1139, 19569]),
  tensor([ 1104,  1107,     0, 10144,  1282,   117,  6124,  2737]),
  tensor([ 118,  170,    0, 2525, 1137, 1133,  112, 6077]),
  tensor([ 1103, 14255,     0,  1524,  1725,  1147,   188,  1114]),
  tensor([ 118, 2572,    0, 1134, 3141, 1106, 3043, 1139]),
  tensor([6159,  165,    0, 1110, 1116, 3740, 1105, 6642]),
  tensor([1983,  183,    0, 1632, 1567, 5878, 1238, 6193]),
  tensor([1457, 2137,    0, 1111, 1122, 1116,  112, 1121]),
  tensor([23783,  2047,     0,  2903,   117,  1

In [60]:
model (**blub)

AttributeError: ignored